In [21]:
import json
from unidecode import unidecode
import polyline
import osmnx as ox
import geopandas as gpd
from shapely.geometry import LineString, Point
from tqdm import tqdm

In [5]:
with open("data/clean/segments.json", "r") as f:
    segments = json.load(f)

In [13]:
regije = {
    "Pomurska": [
        "Apace", "Beltinci", "Cankova", "Crensovci", "Dobrovnik", "Gornja Radgona", "Gornji Petrovci", "Grad",
        "Hodos", "Kobilje", "Krizevci", "Kuzma", "Lendava", "Ljutomer", "Moravske Toplice", "Murska Sobota",
        "Odranci", "Puconci", "Radenci", "Razkrizje", "Rogasovci", "Sveti Jurij ob Scavnici", "Salovci",
        "Tisina", "Turnisce", "Velika Polana", "Verzej"
    ],
    "Podravska": [
        "Benedikt", "Cerkvenjak", "Cirkulane", "Destrnik", "Dornava", "Duplek", "Gorisnica", "Hajdina",
        "Hoce-Slivnica", "Jursinci", "Kidricevo", "Kungota", "Lenart", "Lovrenc na Pohorju", "Majsperk",
        "Makole", "Maribor", "Markovci", "Miklavz na Dravskem polju", "Oplotnica", "Ormoz", "Pesnica",
        "Podlehnik", "Poljcane", "Ptuj", "Race-Fram", "Ruse", "Selnica ob Dravi", "Slovenska Bistrica",
        "Sredisce ob Dravi", "Starse", "Sveta Ana", "Sveta Trojica v Slovenskih goricah",
        "Sveti Andraz v Slovenskih goricah", "Sveti Jurij v Slovenskih goricah", "Sveti Tomaz", "Sentilj",
        "Trnovska vas", "Videm", "Zavrc", "Zetale"
    ],
    "Koroska": [
        "Crna na Koroskem", "Dravograd", "Mezica", "Mislinja", "Muta", "Podvelka", "Prevalje", "Radlje ob Dravi",
        "Ravne na Koroskem", "Ribnica na Pohorju", "Slovenj Gradec", "Vuzenica"
    ],
    "Savinjska": [
        "Bistrica ob Sotli", "Braslovce", "Celje", "Dobje", "Dobrna", "Gornji Grad", "Kozje", "Lasko", "Ljubno",
        "Luce", "Mozirje", "Nazarje", "Podcetrtek", "Polzela", "Prebold", "Radece", "Recica ob Savinji",
        "Rogaska Slatina", "Rogatec", "Slovenske Konjice", "Solcava", "Sentjur", "Smarje pri Jelsah",
        "Smartno ob Paki", "Sostanj", "Store", "Tabor", "Velenje", "Vitanje", "Vojnik", "Vransko", "Zrece",
        "Zalec"
    ],
    "Jugovzhodna Slovenija": [
        "Crnomelj", "Dolenjske Toplice", "Kocevje", "Kostel", "Loski Potok", "Metlika", "Mirna", "Mirna Pec",
        "Mokronog-Trebelno", "Novo mesto", "Osilnica", "Ribnica", "Semic", "Sodrazica", "Straza", "Sentjernej",
        "Sentrupert", "Skocjan", "Smarjeske Toplice", "Trebnje", "Zuzemberk", "Hrastnik", "Trbovlje",
        "Zagorje ob Savi", "Brezice", "Kostanjevica na Krki", "Krsko", "Sevnica"
    ],
    "Osrednjeslovenska": [
        "Borovnica", "Brezovica", "Dobrepolje", "Dobrova-Polhov Gradec", "Dol pri Ljubljani", "Domzale",
        "Grosuplje", "Horjul", "Ig", "Ivancna Gorica", "Kamnik", "Komenda", "Litija", "Ljubljana",
        "Log-Dragomer", "Logatec", "Lukovica", "Medvode", "Menges", "Moravce", "Skofljica", "Smartno pri Litiji",
        "Trzin", "Velike Lasce", "Vodice", "Vrhnika"
    ],
    "Gorenjska": [
        "Bled", "Bohinj", "Cerklje na Gorenjskem", "Gorenja vas-Poljane", "Gorje", "Jesenice", "Jezersko",
        "Kranj", "Kranjska Gora", "Naklo", "Preddvor", "Radovljica", "Sencur", "Skofja Loka", "Trzic",
        "Zelezniki", "Ziri", "Zirovnica"
    ],
    "Goriska": [
        "Ajdovscina", "Bovec", "Brda", "Cerkno", "Idrija", "Kanal", "Kobarid", "Miren-Kostanjevica",
        "Nova Gorica", "Rence-Vogrsko", "Sempeter-Vrtojba", "Tolmin", "Vipava"
    ],
    "Obalno-kraska": [
        "Ankaran", "Divaca", "Hrpelje-Kozina", "Izola", "Komen", "Koper", "Piran", "Sezana"
    ],
    "Notranjsko-kraska": [
        "Bloke", "Cerknica", "Ilirska Bistrica", "Loska dolina", "Pivka", "Postojna"
    ]
}


In [14]:
summ = 0
for regija in regije:
    summ += len(regije[regija])
print(f"Število občin v Sloveniji: {summ}")

Število občin v Sloveniji: 212


In [20]:
obcina_v_regijo = {
    obcina: regija
    for regija, obcine in regije.items()
    for obcina in obcine
}

# Čiščenje občin (odstrani šumnike, uredi pomišljaje)
def normalize_obcina(ime):
    return unidecode(ime).replace(" - ", "-")

# Dodaj regijo za vsako občino v segmentih
for segment in segments:
    segment["regija"] = []
    for obcina in segment["region"]:
        if obcina == "Novo Mesto":
            # Posebna obravnava za Novo Mesto, ker se pojavlja v različnih oblikah
            normalized = "Novo mesto"
        elif obcina == "Nova Goriška":
            # Posebna obravnava za Nova Goriška, ker se pojavlja v različnih oblikah
            normalized = "Nova Gorica"
        elif obcina == "Šentjur pri Celju":
            # Posebna obravnava za Šentjur pri Celju, ker se pojavlja v različnih oblikah
            normalized = "Šentjur"
        else:
            normalized = normalize_obcina(obcina)
        regija = obcina_v_regijo.get(normalized)
        if regija:
            segment["regija"].append(regija)

# Shrani v novo datoteko
with open("data/clean/segments_with_regije.json", "w", encoding="utf-8") as f:
    json.dump(segments, f, indent=4, ensure_ascii=False)    

In [32]:
import requests
from collections import Counter

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

def overpass_surface_for_point(lat, lon, radius=20):
    query = f"""
    [out:json][timeout:25];
    way(around:{radius},{lat},{lon})["highway"]["surface"];
    out tags;
    """
    response = requests.post(OVERPASS_URL, data={"data": query})
    if response.status_code != 200:
        print(f"Napaka Overpass API: {response.status_code}")
        return []

    data = response.json()
    surfaces = []
    for element in data.get("elements", []):
        surface = element.get("tags", {}).get("surface")
        if surface:
            surfaces.append(surface)
    return surfaces

def get_representative_points(polyline_coords, n=5):
    length = len(polyline_coords)
    if length <= n:
        return polyline_coords
    step = length // n
    return [polyline_coords[i*step] for i in range(n)]

def determine_segment_surface(polyline_coords):
    points = get_representative_points(polyline_coords, n=5)
    all_surfaces = []
    for lat, lon in points:
        surfaces = overpass_surface_for_point(lat, lon)
        all_surfaces.extend(surfaces)

    if not all_surfaces:
        return "unknown"

    counter = Counter(all_surfaces)
    most_common_surface, count = counter.most_common(1)[0]
    return most_common_surface

with open("data/clean/segments_with_regije.json", "r", encoding="utf-8") as f:
    segments = json.load(f)

for i, segment in enumerate(segments):
    try:
        polyline_str = segment.get("map", {}).get("polyline")
        if not polyline_str:
            segment["surface"] = "unknown"
            continue
        
        coords = polyline.decode(polyline_str)  # dekodiraj polyline
        surface = determine_segment_surface(coords)
        segment["surface"] = surface
        print(f"Segment {i} - Surface: {surface}")

    except Exception as e:
        print(f"Napaka pri segmentu {i}: {e}")
        segment["surface"] = "unknown"

with open("data/clean/segments_with_surface.json", "w", encoding="utf-8") as f:
    json.dump(segments, f, ensure_ascii=False, indent=4)


Segment 0 - Surface: asphalt
Segment 1 - Surface: asphalt
Segment 2 - Surface: asphalt
Segment 3 - Surface: asphalt
Segment 4 - Surface: asphalt
Segment 5 - Surface: asphalt
Segment 6 - Surface: asphalt
Segment 7 - Surface: asphalt
Segment 8 - Surface: asphalt
Segment 9 - Surface: asphalt
Segment 10 - Surface: paved
Segment 11 - Surface: asphalt
Segment 12 - Surface: asphalt
Segment 13 - Surface: asphalt
Segment 14 - Surface: asphalt
Segment 15 - Surface: asphalt
Segment 16 - Surface: asphalt
Segment 17 - Surface: asphalt
Segment 18 - Surface: asphalt
Segment 19 - Surface: asphalt
Segment 20 - Surface: unknown
Segment 21 - Surface: asphalt
Segment 22 - Surface: asphalt
Segment 23 - Surface: gravel
Segment 24 - Surface: asphalt
Segment 25 - Surface: asphalt
Segment 26 - Surface: asphalt
Segment 27 - Surface: asphalt
Segment 28 - Surface: asphalt
Segment 29 - Surface: gravel
Segment 30 - Surface: unknown
Segment 31 - Surface: asphalt
Segment 32 - Surface: compacted
Segment 33 - Surface: 